<a href="https://colab.research.google.com/github/dadashzadeh/Keyword-Suggestion-Tool/blob/main/Autocomplete_Suggestions_Graph_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install httpx
!!pip install pyvis
!!pip install networkx

['Requirement already satisfied: networkx in /usr/local/lib/python3.10/dist-packages (3.4.2)']

In [4]:
import httpx
from pyvis.network import Network
import networkx as nx
import pandas as pd
from datetime import datetime
import os
from google.colab import files


In [5]:

# Function to fetch autocomplete suggestions using Google’s public autocomplete API
def search_autocomplete(query):
    try:
        url = f"http://suggestqueries.google.com/complete/search?client=firefox&q={query}"
        with httpx.Client() as client:
            response = client.get(url, timeout=5)
            # Parse the response as a JSON object
            suggestions = response.json()[1]  # The second element of the response contains the suggestions
            return suggestions
    except Exception as e:
        print(f"Error fetching autocomplete suggestions for {query}: {e}")
        return []

# Function to recursively add suggestions to the graph
def add_suggestions_to_graph(G, query, data, depth=2, current_depth=1):
    if current_depth > depth:
        return

    # Fetch autocomplete suggestions for the current query
    suggestions = search_autocomplete(query)
    if not suggestions:  # Check if no suggestions were returned
        return

    # Add nodes and edges for each suggestion
    for suggestion in suggestions:
        if not G.has_node(suggestion):  # Avoid adding duplicate nodes
            G.add_node(suggestion)
        if not G.has_edge(query, suggestion):  # Avoid adding duplicate edges
            G.add_edge(query, suggestion)

        # Add to data collection for saving later (must append to the list)
        data.append({"Source": query, "Target": suggestion, "Depth": current_depth})

        # Recursively fetch suggestions for this suggestion (sub-suggestions)
        add_suggestions_to_graph(G, suggestion, data, depth, current_depth + 1)

# Main query to search for autocomplete suggestions
query = "سئو"

# Create a networkx graph
G = nx.Graph()

# Add the main query node
G.add_node(query)

# Initialize data collection as an empty list (IMPORTANT)
data = []  # Ensure this is a list, not an integer

# Add suggestions to the graph recursively up to the desired depth (e.g., depth=2)
add_suggestions_to_graph(G, query, data, depth=3)

# Ensure the graph is not empty
if len(G.nodes) > 1:
    # Create a pyvis Network instance
    net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black', notebook=False, cdn_resources="local")

    # Convert the NetworkX graph to Pyvis graph
    net.from_nx(G)

    # Generate and save the interactive graph as an HTML file
    net.write_html("autocomplete_graph.html")

    # Create a pandas DataFrame from the collected data (edges)
    edges_df = pd.DataFrame(data)

    # Create a pandas DataFrame for the nodes
    nodes_df = pd.DataFrame({"Node": list(G.nodes)})

    curr_datetime = datetime.now().strftime('%Y-%m-%d%H-%M-%S')
    splitted_path = os.path.splitext(f'{query}-.xlsx')
    modified_xlsx_path = splitted_path[0] + curr_datetime + splitted_path[1]

    # Save both nodes and edges to separate sheets in an Excel file
    with pd.ExcelWriter(modified_xlsx_path) as writer:
        nodes_df.to_excel(writer, sheet_name="Nodes", index=False)
        edges_df.to_excel(writer, sheet_name="Edges", index=False)


Download the generated file

In [8]:
files.download(modified_xlsx_path)
files.download("autocomplete_graph.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>